In [1]:
% load_ext autoreload
% autoreload 2
% matplotlib inline

import os
from functools import reduce

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

import sys
sys.path.append('..')
from src.evaluation.evaluator import Evaluator
from src.datasets import SyntheticDataGenerator, MultivariateAnomalyFunction
import main

c:\users\thomas\python\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


TODO:
- XOR (Anomaly -> gleichzeitig)
- Ableitung (Geschwindigkeit vs Beschleunigung)
- Invers

## Plot missing results

In [9]:
datasets = [SyntheticDataGenerator.get(f'extreme_1_missing', 42, missing) for missing in np.linspace(0, 0.9, 5)]
detectors = main.get_detectors()[:1] + main.get_detectors()[3:4]

In [ ]:
seeds = np.random.randint(low=0, high=2 ** 32 - 1, size=2, dtype="uint32")
results = pd.DataFrame()

for i, seed in enumerate(seeds):
    evaluator = Evaluator(datasets if datasets else get_pipeline_datasets(seed), detectors, seed=seed, output_dir='data')
    evaluator.evaluate()
    result = evaluator.benchmarks()
    evaluator.benchmark_results = result
    evaluator.export_results(f'missing-run-{i}-{seed}')
    del evaluator  # Save memory
    results = results.append(result, ignore_index=True)

In [ ]:
avg_results = results.groupby(["dataset", "algorithm"], as_index=False).mean()
std_results = results.groupby(["dataset", "algorithm"]).std(ddof=0)

evaluator = Evaluator(datasets if datasets else get_pipeline_datasets(seed), detectors, seed=42, output_dir='data')
evaluator.benchmark_results = avg_results

In [ ]:
print(f'{"-"*20} Average {"-"*20}')
print(avg_results)
print(f'{"-"*22} Std {"-"*22}')
print(std_results)

In [ ]:
path = evaluator.export_results('missing-final')

In [ ]:
evaluator2 = Evaluator(datasets, detectors, output_dir='data')
evaluator2.import_results(path[5:-4])

In [ ]:
fig = evaluator2.plot_experiment_comparison('Missing')

### Read results from pickle

In [ ]:
pickle_dirs = ['data/exe2']
datasets = [SyntheticDataGenerator.get(f'extreme_1_missing', 42, missing) for missing in np.linspace(0, 0.9, 5)]
detectors = main.get_detectors()
evaluators = []
for dir_ in pickle_dirs:
    for path in os.listdir(os.path.join(dir_, 'evaluators')):
        ev = Evaluator(datasets, detectors, output_dir=dir_)
        ev.import_results(path[:-4])
        evaluators.append(ev)
results = [ev.benchmark_results for ev in evaluators]

In [ ]:
aurocs = [x[['algorithm', 'dataset', 'auroc']] for x in results]
aurocs_df = pd.concat(aurocs, axis=0, ignore_index=True)
auroc_groups = aurocs_df.groupby(['algorithm', 'dataset'])
aurocs_mean = auroc_groups.mean().reset_index()
aurocs_std= auroc_groups.std().reset_index()

In [ ]:
det_names = [str(x) for x in detectors[:]]
ds_names = [str(x) for x in datasets]
fig, axes = plt.subplots(len(det_names), figsize = (8, 5*len(det_names)))
for ax, det in zip(axes.flat, det_names):
    values = aurocs_df[aurocs_df['algorithm'] == det].drop(columns='algorithm')
    ds_groups = values.groupby('dataset')
    ax.boxplot([ds_groups.get_group(x)['auroc'].values for x in ds_names])
    ax.set_title(f'Area under ROC for {det}')
    ax.set_xticklabels([Evaluator.get_key_and_value(x)[1] for x in ds_names])
    ax.set_xlabel('Missing percentage for extreme outliers dataset')
    ax.set_ylabel('AUROC')
    ax.set_ylim((0, 1))
fig.tight_layout()
fig.savefig('boxplots-auroc-datasets.pdf')

In [ ]:
mean_ev = Evaluator(datasets, detectors, output_dir='data')
mean_ev.set_benchmark_results(aurocs_mean)
fig = mean_ev.plot_experiment_comparison('Missing')
fig.savefig('missing-auroc.pdf')

### Evaluate DAGMM

In [ ]:
datasets2 = [SyntheticDataGenerator.get(f'variance_1_missing', 100, 0)]
detectors2 = main.get_detectors()[3:4]
ev2 = Evaluator(datasets2, detectors2, output_dir=dir_)
ev2.evaluate()
result = ev2.benchmarks()
ev2.benchmark_results = result
# evaluator.export_results(f'missing-run-{i}-{seed}')

In [ ]:
ev2.plot_roc_curves()

In [ ]:
print(ev2.benchmark_results)
ev2.plot_scores(9)